In [2]:
!git clone https://github.com/ml-utils/bert-syntax-it.git
!mv ./bert-syntax-it/marvin_linzen_dataset.tsv ./marvin_linzen_dataset.tsv

Cloning into 'bert-syntax-it'...
remote: Enumerating objects: 209, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 209 (delta 95), reused 185 (delta 72), pack-reused 0
Receiving objects: 100% (209/209), 3.12 MiB | 9.40 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [3]:

!ls -la

total 20244
drwxr-xr-x 1 root root     4096 May  4 08:19 .
drwxr-xr-x 1 root root     4096 May  4 08:17 ..
drwxr-xr-x 6 root root     4096 May  4 08:19 bert-syntax-it
drwxr-xr-x 4 root root     4096 Apr 29 03:18 .config
-rw-r--r-- 1 root root 20705886 May  4 08:19 marvin_linzen_dataset.tsv
drwxr-xr-x 1 root root     4096 Apr 29 03:19 sample_data


In [1]:
!pip install folium==0.2.1
!pip install pytorch-pretrained-bert
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 69 kB 3.7 MB/s 
  Created wheel for folium: filename=folium-0.2.1-py3-none-any.whl size=79808 sha256=13cc90c3e878cc1c3032e22e1c8eae5b65956e7458eb47f2c9ed8292db14794c
  Stored in directory: /root/.cache/pip/wheels/9a/f0/3a/3f79a6914ff5affaf50cabad60c9f4d565283283c97f0bdccf
Successfully built folium
  Attempting uninstall: folium
    Found existing installation: folium 0.8.3
    Uninstalling folium-0.8.3:
      Successfully uninstalled folium-0.8.3
     |████████████████████████████████| 123 kB 5.3 MB/s 
     |████████████████████████████████| 132 kB 39.0 MB/s 
     |████████████████████████████████| 79 kB 7.9 MB/s 
     |████████████████████████████████| 8.7 MB 47.8 MB/s 
     |████████████████████████████████| 138 kB 48.2 MB/s 
     |████████████████████████████████| 127 kB 48.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
     

In [5]:
import os.path
from collections import Counter

from pytorch_pretrained_bert import BertForMaskedLM,tokenization
from torch.nn.functional import softmax
import torch
import argparse, sys
import csv

In [6]:

def tokenize_sentence(tokenizer,sent):
    print(f'sent: {sent}')
    pre,target,post=sent.split('***')
    print(f'pre: {pre}, target: {target}, post: {post}')
    if 'mask' in target.lower():
        target=['[MASK]']
    else:
        target=tokenizer.tokenize(target)

    # todo, fixme: the vocabulary of the pretrained model from Kaj does not have entries for CLS, UNK
    # fixme: tokenizer.tokenize(pre), does not recognize the words
    tokens=['[CLS]']+tokenizer.tokenize(pre)  # tokens = tokenizer.tokenize(pre)
    target_idx=len(tokens)
    print(f'target_idx: {target_idx}')
    tokens+=target+tokenizer.tokenize(post)+['[SEP]']
    print(f'tokens {tokens}')
    return tokens, target_idx


def get_sentence_probs_from_word_ids(bert, tokenizer, sentence_ids, masked_words_ids, masked_word_idx):
    tens=torch.LongTensor(sentence_ids).unsqueeze(0)

    res_unsliced = bert(tens)
    res=res_unsliced[0, masked_word_idx]

    # todo: produce probabilities not with softmax (not using an exponential, to avoiding the maximization of top results),
    #  then compare these probailities with the softmax ones, expecially for ungrammatical sentences
    res_softmax = softmax(res.detach(), -1)
    #RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

    topk_probs, topk_ids = torch.topk(res_softmax, 10)  # todo: get ids/indexes, not their probability value
    topk_tokens = convert_ids_to_tokens(tokenizer, topk_ids)

    scores = res[masked_words_ids]
    probs = [float(x) for x in scores]
    scores_softmax = res_softmax[masked_words_ids]
    probs_softmax = [float(x) for x in scores_softmax]

    return probs_softmax  # probs


def estimate_sentence_probability_from_text(bert, tokenizer, sentence):
    tokens = tokenizer.tokenize(sentence)
    sentence_ids = tokenizer.convert_tokens_to_ids(tokens)
    return estimate_sentence_probability(bert, tokenizer, sentence_ids)


def estimate_sentence_probability(bert, tokenizer, sentence_ids):
    # iterate for each word, mask it and get the probability
    # sum the logs of the probabilities

    MASK_ID = tokenizer.convert_tokens_to_ids(['[MASK]'])[0]
    CLS_ID = tokenizer.convert_tokens_to_ids(['[CLS]'])[0]
    SEP_ID = tokenizer.convert_tokens_to_ids(['[SEP]'])[0]

    sentence_prob_estimate = 0
    for masked_index in range(len(sentence_ids)):
        masked_sentence_ids = sentence_ids.copy()
        masked_word_id = masked_sentence_ids[masked_index]
        masked_sentence_ids[masked_index] = MASK_ID
        masked_sentence_ids = [CLS_ID] + masked_sentence_ids + [SEP_ID]
        print(f'testing {convert_ids_to_tokens(tokenizer, [masked_word_id])} '
              f'at {masked_index+1} in sentence {convert_ids_to_tokens(tokenizer, masked_sentence_ids)}')
        probability_of_this_masking = get_sentence_probs_from_word_ids(bert, tokenizer, masked_sentence_ids,
                                                                     [masked_word_id], masked_index+1)
        sentence_prob_estimate += np.log(probability_of_this_masking[0])

    # todo: also alternative method with formula from paper balanced on sentence lenght

    return np.exp(sentence_prob_estimate)


def get_probs_for_words(bert, tokenizer, sent, w1, w2):
    tokens, masked_word_idx = tokenize_sentence(tokenizer, sent)

    sentence_ids=tokenizer.convert_tokens_to_ids(tokens)
    try:
        masked_words_ids=tokenizer.convert_tokens_to_ids([w1,w2])
    except KeyError:
        print("skipping",w1,w2,"bad wins")
        return None

    return get_sentence_probs_from_word_ids(bert, tokenizer, sentence_ids, masked_words_ids, masked_word_idx)


def convert_ids_to_tokens(tokenizer, ids):
    """Converts a sequence of ids in wordpiece tokens using the vocab."""
    tokens = []
    for i in ids:
        if torch.is_tensor(i):
            i = i.item()
        # print(f"id: {i}, type: {type(i)}")
        try:
            tokens.append(tokenizer.ids_to_tokens[i])
        except:
            print(f"Unable to find id {i} {type(i)} in the vocabulary")

    return tokens


def load_it():
    cc = Counter()
    # note: I edited the LM_Syneval/src/make_templates.py script, and run "python LM_Syneval/src/make_templates.py LM_Syneval/data/templates/ > marvin_linzen_dataset.tsv"
    out = []
    for line in open("it_dataset.tsv"):
        case = line.strip().split("\t")
        cc[case[1]]+=1
        g,ug = case[-2],case[-1]
        g = g.split()
        ug = ug.split()
        assert(len(g)==len(ug)),(g,ug)
        diffs = [i for i,pair in enumerate(zip(g,ug)) if pair[0]!=pair[1]]
        if (len(diffs)!=1):
            #print(diffs)
            #print(g,ug)
            continue
        assert(len(diffs)==1),diffs
        gv=g[diffs[0]]   # good
        ugv=ug[diffs[0]] # bad
        g[diffs[0]]="***mask***"
        g.append(".")
        out.append((case[0],case[1]," ".join(g),gv,ugv))
    return out

def load_marvin():
    cc = Counter()
    # note: I edited the LM_Syneval/src/make_templates.py script, and run "python LM_Syneval/src/make_templates.py LM_Syneval/data/templates/ > marvin_linzen_dataset.tsv"
    out = []
    for line in open("marvin_linzen_dataset.tsv"):
        case = line.strip().split("\t")
        cc[case[1]]+=1
        g,ug = case[-2],case[-1]
        g = g.split()
        ug = ug.split()
        assert(len(g)==len(ug)),(g,ug)
        diffs = [i for i,pair in enumerate(zip(g,ug)) if pair[0]!=pair[1]]
        if (len(diffs)!=1):
            #print(diffs)
            #print(g,ug)
            continue    
        assert(len(diffs)==1),diffs
        gv=g[diffs[0]]   # good
        ugv=ug[diffs[0]] # bad
        g[diffs[0]]="***mask***"
        g.append(".")
        out.append((case[0],case[1]," ".join(g),gv,ugv))
    return out

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def eval_it(bert,tokenizer):
    o = load_it()
    print(len(o), file=sys.stderr)
    from collections import defaultdict
    import time
    rc = defaultdict(Counter)
    tc = Counter()
    start = time.time()
    print(f'{bcolors.WARNING}{len(o)} sentences to process..{bcolors.ENDC}')
    for i, (case, tp, s, good_word, bad_word) in enumerate(o):
        ps = get_probs_for_words(bert, tokenizer, s, good_word, bad_word)
        if ps is None: ps = [0, 1]
        gp = ps[0]
        bp = ps[1]
        print(gp > bp, case, tp, good_word, bad_word, s)
        if i % 100 == 0:
            print(f'{bcolors.WARNING}{i}{bcolors.ENDC}')
            print(i, time.time() - start, file=sys.stderr)
            start = time.time()
            sys.stdout.flush()

def eval_marvin(bert,tokenizer):
    o = load_marvin()
    print(len(o),file=sys.stderr)
    from collections import defaultdict
    import time
    rc = defaultdict(Counter)
    tc = Counter()
    start = time.time()
    for i,(case,tp,s,g,b) in enumerate(o):
        ps = get_probs_for_words(bert,tokenizer,s,g,b)
        if ps is None: ps = [0,1]
        gp = ps[0]
        bp = ps[1]
        print(gp>bp,case,tp,g,b,s)
        if i % 100==0:
            print(i,time.time()-start,file=sys.stderr)
            start=time.time()
            sys.stdout.flush()

def eval_lgd(bert,tokenizer):
    for i,line in enumerate(open("lgd_dataset_with_is_are.tsv",encoding="utf8")):
        na,_,masked,good,bad = line.strip().split("\t")
        ps = get_probs_for_words(bert,tokenizer,masked,good,bad)
        if ps is None: continue
        gp = ps[0]
        bp = ps[1]
        print(str(gp>bp),na,good,gp,bad,bp,masked.encode("utf8"),sep=u"\t")
        if i%100 == 0:
            print(i,file=sys.stderr)
            sys.stdout.flush()


def read_gulordava():
    rows = csv.DictReader(open("generated.tab",encoding="utf8"),delimiter="\t")
    data=[]
    for row in rows:
        row2=next(rows)
        assert(row['sent']==row2['sent'])
        assert(row['class']=='correct')
        assert(row2['class']=='wrong')
        sent = row['sent'].lower().split()[:-1] # dump the <eos> token.
        good_form = row['form']
        bad_form  = row2['form']
        sent[int(row['len_prefix'])]="***mask***"
        sent = " ".join(sent)
        data.append((sent,row['n_attr'],good_form,bad_form))
    return data


def eval_gulordava(bert,tokenizer):
    for i,(masked,natt,good,bad) in enumerate(read_gulordava()):
        if good in ["is","are"]:
            print("skipping is/are")
            continue
        ps = get_probs_for_words(bert,tokenizer,masked,good,bad)
        if ps is None: continue
        gp = ps[0]
        bp = ps[1]
        print(str(gp>bp),natt,good,gp,bad,bp,masked.encode("utf8"),sep=u"\t")
        if i%100 == 0:
            print(i,file=sys.stderr)
            sys.stdout.flush()

# choose_eval()


def init_bert_model(model_name, dict_name=None):
    # model_name = 'bert-large-uncased'
    #if 'base' in sys.argv: model_name = 'bert-base-uncased'
    print(f'model_name: {model_name}')
    print("loading model:", model_name, file=sys.stderr)
    bert = BertForMaskedLM.from_pretrained(model_name)
    print("bert model loaded, getting the tokenizer..")

    if dict_name is None:
        vocab_filepath = model_name
    else:
        vocab_filepath = os.path.join(model_name, 'dict.txt')
    tokenizer = tokenization.BertTokenizer.from_pretrained(vocab_filepath)

    print("tokenizer ready.")

    bert.eval()
    return bert, tokenizer


def run_eval(eval_suite, bert, tokenizer):
    print(f'running eval, eval_suite: {eval_suite}')
    if 'it' == eval_suite:
        eval_it(bert, tokenizer)
    elif 'marvin' == eval_suite:
        eval_marvin(bert,tokenizer)
    elif 'gul' == eval_suite:
        eval_gulordava(bert,tokenizer)
    else:
        eval_lgd(bert,tokenizer)


def arg_parse():
    print('parsing args..')
    # Python program to demonstrate
    # command line arguments

    import getopt, sys

    # Remove 1st argument from the
    # list of command line arguments
    argumentList = sys.argv[1:]

    options = "be:"

    # Long options
    long_options = ["bert_model", "eval_suite"]

    DEFAULT_MODEL = 'bert-large-uncased'
    DEFAULT_EVAL_SUITE = 'lgd'
    model_name = DEFAULT_MODEL
    eval_suite = DEFAULT_EVAL_SUITE

    try:
        # Parsing argument
        print(f'argumentList: {argumentList}')

        # checking each argument
        for arg_idx, currentArgument  in enumerate(argumentList):
            print(f'persing currentArgument {currentArgument}')
            if currentArgument in ("-h", "--Help"):
                print("Displaying Help")

            elif currentArgument in ("-b", "--bert_model"):

                argValue = argumentList[arg_idx+1]
                print(f'currentArgument: {currentArgument}, argValue: {argValue}')
                if argValue == 'base':
                    model_name = 'bert-base-uncased'
                else:
                    model_name = argValue
                    print(f'set model_name: {model_name}')

            elif currentArgument in ("-e", "--eval_suite"):
                argValue = argumentList[arg_idx + 1]
                print(f'currentArgument: {currentArgument}, argValue: {argValue}')
                eval_suite = argValue

    except getopt.error as err:
        # output error, and return with an error code
        print(str(err))

    print(f'model_name {model_name}, eval_suite {eval_suite}')
    return model_name, eval_suite


import numpy as np
from scipy.special import softmax
def bert_get_logprobs(tokenize_input, tokenize_context, model, tokenizer, device, use_context=False):
    batched_indexed_tokens = []
    batched_segment_ids = []

    if not use_context:
        tokenize_combined = ["[CLS]"] + tokenize_input + ["[SEP]"]
    else:
        tokenize_combined = ["[CLS]"] + tokenize_context + tokenize_input + ["[SEP]"]

    for i in range(len(tokenize_input)):
        # Mask a token that we will try to predict back with `BertForMaskedLM`
        masked_index = i + 1 + (len(tokenize_context) if use_context else 0)
        tokenize_masked = tokenize_combined.copy()
        tokenize_masked[masked_index] = '[MASK]'
        # unidir bert
        # for j in range(masked_index, len(tokenize_combined)-1):
        #    tokenize_masked[j] = '[MASK]'

        # Convert token to vocabulary indices
        indexed_tokens = tokenizer.convert_tokens_to_ids(tokenize_masked)
        # Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
        segment_ids = [0] * len(tokenize_masked)

        batched_indexed_tokens.append(indexed_tokens)
        batched_segment_ids.append(segment_ids)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor(batched_indexed_tokens, device=device)
    segment_tensor = torch.tensor(batched_segment_ids, device=device)

    # Predict all tokens
    with torch.no_grad():
        outputs = model(tokens_tensor, token_type_ids=segment_tensor)
        predictions = outputs[0]

    # go through each word and sum their logprobs
    lp = 0.0
    for i in range(len(tokenize_input)):
        masked_index = i + 1 + (len(tokenize_context) if use_context else 0)
        predicted_score = predictions[i, masked_index]
        predicted_prob = softmax(predicted_score.cpu().numpy())
        lp += np.log(predicted_prob[tokenizer.convert_tokens_to_ids([tokenize_combined[masked_index]])[0]])

    return lp


def get_masked_word_probability(bert, tokenizer):
    return 0


def custom_eval(sentence, bert, tokenizer):
    bert, tokenizer = init_bert_model('bert-base-uncased')

    compare_tokens, compare_target_idx = tokenize_sentence(tokenizer, "What is ***your*** name?")

    bare_sentence_tokens = tokenizer.tokenize(sentence)

    paper_logprobs = bert_get_logprobs(bare_sentence_tokens, None, bert, tokenizer, device=None)

    tokens_list = ['[CLS]'] + bare_sentence_tokens + ['[SEP]']
    target_idx = 3
    masked_word = tokens_list[target_idx]
    tokens_list[target_idx] = '[MASK]'

    print(f'tokens: {tokens_list}, masked_word: {masked_word}')
    print(f'compare_tokens: {compare_tokens}')

    input_ids = tokenizer.convert_tokens_to_ids(tokens_list)

    try:
        masked_word_id = tokenizer.convert_tokens_to_ids([masked_word])
    except KeyError:
        print(f"unable to convert {masked_word} to id")
        return None
    tens = torch.LongTensor(input_ids).unsqueeze(0)

    res_unsliced = bert(tens)
    res=res_unsliced[0, target_idx]

    # res=torch.nn.functional.softmax(res,-1)

    pred = bert("What is [MASK] name?")

    # Set the maximum sequence length.
    MAX_LEN = 128
    # Pad our input tokens
    input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                              maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
    # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")


def main():
    print('main')
    model_name, eval_suite = arg_parse()
    model_name = 'bert-base-uncased'  # NB bert large uncased is about 1GB
    eval_suite = 'it'
    bert, tokenizer = init_bert_model(model_name)
    if tokenizer is None:
        print('error, tokenizer is null')
        return

    # run_eval(eval_suite, bert, tokenizer)
    prob1 = estimate_sentence_probability_from_text(bert, tokenizer, 'What is your name?')
    prob2 = estimate_sentence_probability_from_text(bert, tokenizer, 'What is name your?')
    print(f'prob1: {prob1}, prob2: {prob2}')
    #eval_it(bert, tokenizer)
    #custom_eval("What is your name?", bert, tokenizer)


In [7]:
main()

main
parsing args..
argumentList: ['-f', '/root/.local/share/jupyter/runtime/kernel-5dfbee09-a781-45ea-9ccc-a931b3177835.json']
persing currentArgument -f
persing currentArgument /root/.local/share/jupyter/runtime/kernel-5dfbee09-a781-45ea-9ccc-a931b3177835.json
model_name bert-large-uncased, eval_suite lgd
model_name: bert-base-uncased


loading model: bert-base-uncased
100%|██████████| 407873900/407873900 [00:17<00:00, 22690590.47B/s]


bert model loaded, getting the tokenizer..


100%|██████████| 231508/231508 [00:00<00:00, 1978723.31B/s]


tokenizer ready.
testing ['what'] at 1 in sentence ['[CLS]', '[MASK]', 'is', 'your', 'name', '?', '[SEP]']
testing ['is'] at 2 in sentence ['[CLS]', 'what', '[MASK]', 'your', 'name', '?', '[SEP]']
testing ['your'] at 3 in sentence ['[CLS]', 'what', 'is', '[MASK]', 'name', '?', '[SEP]']
testing ['name'] at 4 in sentence ['[CLS]', 'what', 'is', 'your', '[MASK]', '?', '[SEP]']
testing ['?'] at 5 in sentence ['[CLS]', 'what', 'is', 'your', 'name', '[MASK]', '[SEP]']
testing ['what'] at 1 in sentence ['[CLS]', '[MASK]', 'is', 'name', 'your', '?', '[SEP]']
testing ['is'] at 2 in sentence ['[CLS]', 'what', '[MASK]', 'name', 'your', '?', '[SEP]']
testing ['name'] at 3 in sentence ['[CLS]', 'what', 'is', '[MASK]', 'your', '?', '[SEP]']
testing ['your'] at 4 in sentence ['[CLS]', 'what', 'is', 'name', '[MASK]', '?', '[SEP]']
testing ['?'] at 5 in sentence ['[CLS]', 'what', 'is', 'name', 'your', '[MASK]', '[SEP]']
prob1: 0.289340392786225, prob2: 7.71620475199385e-11
